In [36]:
import os
%load_ext autoreload
%autoreload 2
import pandas as pd
from toolsets.setup import set_workspace, get_dirs
from feature_normalization.rt_normalization import rt_normalization
from toolsets.feature_alignment import initilize_pmz_rt_list, align, find_istd
from toolsets.file_io import get_file_list
from toolsets.search import quick_search_values, string_search

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
set_workspace('/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica')
# this is the master workspace, all other directories will be created within this workspace

the workspace exists!
set up complete


In [5]:
dirs =get_dirs('/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica', mode='pos')
# a lot of data are mode specific, so make sure to include 'mode' argument in there

In [2]:
istd_info = pd.read_csv('/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/istd_info.csv')

In [24]:
dirs

['/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/alignment_result/pos',
 '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/bioactivity',
 '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/mzml/pos',
 '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/normalized_peak_list/pos',
 '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/peak_list/pos',
 '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/results/pos',
 '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/sirius_files/pos']

In [29]:
dirs[3]

'/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/normalized_peak_list/pos'

In [20]:
rt_normalization(peak_list_dir=dirs[4], rt_seed='01090_X0_P', istd_info=istd_info, normalized_peak_list_dir=dirs[3], result_dir=dirs[5])

100%|██████████| 10/10 [01:28<00:00,  8.80s/it]


In [31]:
file_list =get_file_list(dirs[3], '.csv', with_tail=True)

In [33]:
file_list

['01090_X0_P.csv',
 '01090_X1_P.csv',
 '01090_X2_P.csv',
 '01090_X3_P.csv',
 '01090_X4_P.csv',
 '01090_X5_P.csv',
 '01090_X6_P.csv',
 '01090_X7_P.csv',
 '01090_X8_P.csv',
 '01090_X9_P.csv']

In [65]:
master_list = initilize_pmz_rt_list([os.path.join(dirs[3], x) for x in file_list], mode = 'inclusive')

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/Myristica/normalized_peak_list/pos/01090_X0_P.csv'

In [37]:
alignment = align([os.path.join(dirs[3], x) for x in file_list], master_list)
alignment_istd = find_istd(alignment, istd_info)
alignment_istd = string_search(alignment_istd, 'isotopic_state', 'M + 0')
alignment_istd.drop(['isotopic_state'], axis = 1, inplace = True)
alignment_istd.to_csv(os.path.join(dirs[0], 'alignment_result_istd.csv'), index = False)

100%|██████████| 10/10 [01:18<00:00,  7.81s/it]


ValueError: shape mismatch: value array of shape (17,) could not be broadcast to indexing result of shape (19,)

In [46]:
alignment = alignment.copy()
istd_idx = []
for index, row in istd_info.iterrows():
    # mz =
    # rt = row['RT_suggested']
    feature_mz_search = quick_search_values(alignment, 'pmz', row['Precursor m/z']-0.005, row['Precursor m/z']+0.005, ifsorted=False)
    feature_mzrt_search = quick_search_values(feature_mz_search, 'rt', row['RT_suggested']-10/60, row['RT_suggested']+10/60, ifsorted=False)
    if len(feature_mzrt_search.index)>1:
        print(index)
    istd_idx.extend(feature_mzrt_search.index)

1
16


In [52]:
row = istd_info.iloc[16]

In [58]:
from toolsets.feature_alignment import find_feature

In [59]:
row

compound_name       C20-APS
mono_mass        439.312015
RT_suggested           8.29
Precursor m/z    440.319291
Name: 16, dtype: object

In [61]:
row['Precursor m/z']

440.319291

In [62]:
row['RT_suggested']

8.29

In [63]:
find_feature(alignment, row['Precursor m/z'], row['RT_suggested'], mz_column='pmz', rt_column='rt', rt_offset=10)

,pmz,rt,01090_X0_P,01090_X1_P,01090_X2_P,01090_X3_P,01090_X4_P,01090_X5_P,01090_X6_P,01090_X7_P,01090_X8_P,01090_X9_P,isotopic_state
13353,440.3194,8.327345,11186299,10871917,11640460,12259379,10674943,20150348,9093569,8079028,6159597,7006314,M + 0
13354,440.3195,8.327345,11186299,10871917,11640460,12259379,10674943,20150348,9093569,8079028,6159597,7006314,M + 0
